# Serverless

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing_model.h5 --quiet

In [4]:
model = keras.models.load_model('clothing_model.h5')